<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')
    
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В датасете 3214 строк, 5 колонок, данные без пропусков, все колонки нужных типов и с корректными названиями. С виду датасет не нуждается в предобработке, качество данных достаточное для проведения исследования.

## Разбейте данные на выборки

Разбиваем данные на 3 выборки, руководствуясь эмпирическим правилом:
* тестовая выборка - 20%
* обучающая выборка ~ 60%
* валидационная выборка ~ 20%


In [2]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_test, target_train, target_test = (
    train_test_split(features, target, test_size=0.2, random_state=12345)
)
features_train, features_valid, target_train, target_valid = (
    train_test_split(features_train, target_train, test_size=0.25, random_state=12345)
)
len(features_train), len(features_test), len(features_valid)

(2056, 643, 515)

In [ ]:
#Kод ревьюера
!pip install fast_ml
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'is_ultra', 
                                                                            train_size=0.6, valid_size=0.2, test_size=0.2)



## Исследуйте модели

У нас задача классификации. Проверим три модели: Решающее дерево, Случайный лес, Логистическая регрессия. Посчитаем их accuracy и найдем оптимальные гиперпараметры

In [3]:
#Подбор гиперпараметров для дерева решений
best_result = 0
best_model_decision_tree = None
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(max_depth = depth, random_state=12345)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_decision_tree = model
        best_result = result
        best_depth = depth
print(f'Дерево решений (depth = {best_depth}): accuracy = {best_result}')
   
#Случайный лес
best_result = 0
best_est = 0
best_model_random_forest  = None
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_random_forest = model
        best_est = est
        best_result = result

print(f'Случайный лес (n_estimators = {best_est}): accuracy = {best_result}')
    
#Логистическая регрессия
model_logistic_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_logistic_regression.fit(features_valid, target_valid)
best_result = model_logistic_regression.score(features_test, target_test)

print(f'Логистическая регрессия: accuracy = {best_result}')

Дерево решений (depth = 5): accuracy = 0.7572815533980582
Случайный лес (n_estimators = 9): accuracy = 0.7708737864077669
Логистическая регрессия: accuracy = 0.6936236391912908


Лучше всего себя показала модель на основе алгоритма "случайный лес"

## Проверьте модель на тестовой выборке

In [4]:
predictions = model.predict(features_test)
result = accuracy_score(target_test,predictions).round(3)
print(f'Проверка на тестовой выборке: accuracy = {result}')

Проверка на тестовой выборке: accuracy = 0.774


## (бонус) Проверьте модели на адекватность

In [5]:
#Заполним случайными значениями
np.random.seed(42)
target_test_rand = pd.Series(np.random.randint(0, 2, len(target_test)))
print("Заполнение случайными значениями: accuracy =", accuracy_score(target_test, target_test_rand).round(3))

Заполнение случайными значениями: accuracy = 0.496


Заполнение данными случайнами значениями дало значение accuracy = 0.496, что говорит о том, что наша модель прошла проверку на вменяемость (sanity check)

In [3]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [4]:
df.shape[0]

3214

## Выводы

Создана наиболее оптимальная модель машинного обучения для рекомендации тарифа абонентов сотового оператора "Мегалайн". Проанализировано три различных алгоритма машинного обучения с различными гиперпараметрами. В итоге выбрана модель на основе алгоритма "случайный лес" с n_estimators = 9. Модель показала свою адекватность по сравнению со случайной, процент правильных ответов (accuracy) у нее составляет 77,4%